In [1]:
from numpy import *
from pathlib import Path

In [2]:
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float,curLine))
        dataMat.append(fltLine)
    return dataMat

In [14]:
def distEclud(vecA,vecB):
    return sqrt(sum(power(vecA - vecB,2)))

In [6]:
def randCent(dataSet,k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k,1)
    return centroids

In [7]:
data_path = Path('D:\python_algorithm\machinelearinginaction\《机器学习实战》Python3代码\Ch10')

In [11]:
dataMat = mat(loadDataSet(data_path / 'testSet.txt'))

In [16]:
#K-均值聚类算法

In [21]:
def kMeans(dataSet,k,distMeas=distEclud,createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))    #用于记录每一行所属的类别(簇分配的结果)
    centroids = createCent(dataSet,k)
    
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = float(inf)
            minIndex = -1
            
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2      #重新分配
        print(centroids)
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = mean(ptsInClust,axis=0)
            
    return centroids,clusterAssment

In [22]:
myCentroids,clusterAssing = kMeans(dataMat,4)

[[-1.56046662 -3.64029922]
 [ 0.59060624 -2.96110831]
 [-0.42020708 -1.37579214]
 [ 1.70398172  2.21481441]]
[[-3.38237045 -2.9473363 ]
 [ 2.71433141 -3.06923671]
 [-2.7037348   1.395213  ]
 [ 0.70351032  2.85504687]]
[[-3.38237045 -2.9473363 ]
 [ 2.80293085 -2.7315146 ]
 [-2.768021    2.65028438]
 [ 1.98283629  3.1465235 ]]
[[-3.38237045 -2.9473363 ]
 [ 2.80293085 -2.7315146 ]
 [-2.46154315  2.78737555]
 [ 2.6265299   3.10868015]]


In [28]:
#二分K-均值算法
#为了克服k-均值算法收敛于局部最小值的问题。首先将所有点作为一个簇，然后将该簇一分为二。之后选择其中一个簇进行划分（该簇可以最\
#大程度降低误差平方和的值

In [57]:
def biKmeans(dataSet,k,distMeas=distEclud):
    m = shape(dataSet)[0]
    
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet,axis=0).tolist()[0]
    
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
        
    while len(centList) < k:
        lowestSSE = float(inf)
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A == i)[0],:]
            centroidMat,splitClustAss = \
            kMeans(ptsInCurrCluster,2,distMeas)
            sseSplit = sum(splitClustAss[:,1])
            
            sseNotSplit = \
            sum(clusterAssment[nonzero(clusterAssment[:,0].A != i)[0],1])
            
            if sseSplit + sseNotSplit < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList)  #更新簇的分配结果
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
    
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]
        centList.append(bestNewCents[1,:].tolist()[0])
        
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return mat(centList),clusterAssment

In [58]:
dataMat3 = mat(loadDataSet(data_path / 'testSet2.txt'))

In [59]:
centList,myNewAssments = biKmeans(dataMat3,3)

[[ 0.91975193 -3.02742756]
 [-1.72960442  0.23983441]]
[[ 0.00853819 -2.92864213]
 [-0.21818127  2.73586548]]
[[-0.45965615 -2.7782156 ]
 [-0.00675605  3.22710297]]
[[ 0.8357105  -3.2406631 ]
 [-0.64986685 -1.83969857]]
[[ 0.35496167 -3.36033556]
 [-1.12616164 -2.30193564]]
[[0.45043444 3.86270899]
 [4.01917379 3.7152858 ]]
[[-2.04192752  3.38503076]
 [ 3.3851964   2.96389   ]]
[[-2.94737575  3.3263781 ]
 [ 2.93386365  3.12782785]]


In [60]:
import urllib
import json
def geoGrab(stAddress, city):
    apiStem = 'http://where.yahooapis.com/geocode?'  #create a dict and constants for the goecoder
    params = {}
    params['flags'] = 'J'#JSON return type
    params['appid'] = 'aaa0VN6k'
    params['location'] = '%s %s' % (stAddress, city)
    url_params = urllib.parse.urlencode(params)
    yahooApi = apiStem + url_params      #print url_params
    print(yahooApi)
    c=urllib.request.urlopen(yahooApi)
    return json.loads(c.read())

from time import sleep
def massPlaceFind(fileName):
    fw = open('places.txt', 'w')
    for line in open(fileName).readlines():
        line = line.strip()
        lineArr = line.split('\t')
        retDict = geoGrab(lineArr[1], lineArr[2])
        if retDict['ResultSet']['Error'] == 0:
            lat = float(retDict['ResultSet']['Results'][0]['latitude'])
            lng = float(retDict['ResultSet']['Results'][0]['longitude'])
            print("%s\t%f\t%f" % (lineArr[0], lat, lng))
            fw.write('%s\t%f\t%f\n' % (line, lat, lng))
        else: print("error fetching")
        sleep(1)
    fw.close()
    
def distSLC(vecA, vecB):#Spherical Law of Cosines
    a = sin(vecA[0,1]*pi/180) * sin(vecB[0,1]*pi/180)
    b = cos(vecA[0,1]*pi/180) * cos(vecB[0,1]*pi/180) * \
                      cos(pi * (vecB[0,0]-vecA[0,0]) /180)
    return arccos(a + b)*6371.0 #pi is imported with numpy

import matplotlib
import matplotlib.pyplot as plt
def clusterClubs(numClust=5):
    datList = []
    for line in open('places.txt').readlines():
        lineArr = line.split('\t')
        datList.append([float(lineArr[4]), float(lineArr[3])])
    datMat = mat(datList)
    myCentroids, clustAssing = biKmeans(datMat, numClust, distMeas=distSLC)
    fig = plt.figure()
    rect=[0.1,0.1,0.8,0.8]
    scatterMarkers=['s', 'o', '^', '8', 'p', \
                    'd', 'v', 'h', '>', '<']
    axprops = dict(xticks=[], yticks=[])
    ax0=fig.add_axes(rect, label='ax0', **axprops)
    imgP = plt.imread('Portland.png')
    ax0.imshow(imgP)
    ax1=fig.add_axes(rect, label='ax1', frameon=False)
    for i in range(numClust):
        ptsInCurrCluster = datMat[nonzero(clustAssing[:,0].A==i)[0],:]
        markerStyle = scatterMarkers[i % len(scatterMarkers)]
        ax1.scatter(ptsInCurrCluster[:,0].flatten().A[0], ptsInCurrCluster[:,1].flatten().A[0], marker=markerStyle, s=90)
    ax1.scatter(myCentroids[:,0].flatten().A[0], myCentroids[:,1].flatten().A[0], marker='+', s=300)
    plt.show()

In [63]:
# geoResults= geoGrab('1 VA Center','Augusta, ME')  ##抓取失败，因为yahoo的api停止运行